In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fantasy-football/master_team_list.csv
/kaggle/input/fantasy-football/cleaned_merged_seasons.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/kaggle/input/fantasy-football/cleaned_merged_seasons.csv')

/tmp/ipykernel_37/56455962.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/fantasy-football/cleaned_merged_seasons.csv')


In [4]:
df.head()

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,...,2.0,0.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,...,1.0,0.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,...,3.0,23.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,...,1.0,0.0,0,0,0,0,50,False,0,1
4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,...,1.0,0.0,1,0,0,0,45,True,1,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96169 entries, 0 to 96168
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season_x           96169 non-null  object 
 1   name               96169 non-null  object 
 2   position           96169 non-null  object 
 3   team_x             76317 non-null  object 
 4   assists            96169 non-null  int64  
 5   bonus              96169 non-null  int64  
 6   bps                96169 non-null  int64  
 7   clean_sheets       96169 non-null  int64  
 8   creativity         96169 non-null  float64
 9   element            96169 non-null  int64  
 10  fixture            96169 non-null  int64  
 11  goals_conceded     96169 non-null  int64  
 12  goals_scored       96169 non-null  int64  
 13  ict_index          96169 non-null  float64
 14  influence          96169 non-null  float64
 15  kickoff_time       96169 non-null  object 
 16  minutes            961

In [11]:
df.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,96169.000000,96169.000000,96169.000000,96169.000000,96169.000000,96169.000000,96169.000000,96169.000000,96169.000000,96169.000000,...,96169.000000,96169.000000,96169.000000,96169.000000,9.616900e+04,9.616900e+04,9.616900e+04,96169.000000,96169.000000,96169.000000
mean,0.041042,0.108715,6.080275,0.106739,4.734000,327.063264,198.413824,0.495887,0.045514,1.715793,...,1.265158,1.516393,5.287598,1.379270,1.677100e+03,1.454716e+04,1.286945e+04,51.352390,0.053167,20.772099
std,0.215378,0.492334,9.850595,0.308783,10.689879,194.313766,108.884200,0.974269,0.233644,3.042551,...,1.229086,1.349184,13.368420,2.542535,6.581628e+04,5.725343e+04,4.840128e+04,12.667432,0.224367,10.941742
min,0.000000,0.000000,-21.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-7.000000,-2.180978e+06,0.000000e+00,0.000000e+00,37.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,159.000000,105.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-1.374000e+03,4.600000e+01,1.280000e+02,44.000000,0.000000,11.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,318.000000,203.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-5.800000e+01,4.380000e+02,1.104000e+03,48.000000,0.000000,22.000000
75%,0.000000,0.000000,10.000000,0.000000,2.400000,486.000000,293.000000,1.000000,0.000000,2.500000,...,2.000000,2.000000,2.000000,2.000000,7.900000e+01,5.042000e+03,7.562000e+03,54.000000,0.000000,30.000000
max,4.000000,3.000000,128.000000,1.000000,170.900000,778.000000,380.000000,9.000000,4.000000,32.800000,...,7.000000,9.000000,186.000000,29.000000,1.983733e+06,2.104464e+06,2.233619e+06,133.000000,1.000000,38.000000


In [12]:
print(len(df[df['round']!=df['GW']]))

0


In [21]:
# 1. Create a Name-to-ID Map
# We group by the opponent's name and find the most frequent (mode) opponent ID associated with it.
# This creates a reliable 1:1 mapping (Team Name -> Team ID).
name_to_id_map = df.groupby('opp_team_name')['opponent_team'].agg(
    lambda x: x.mode()[0] if not x.mode().empty else np.nan
).to_dict()

# 2. Group Fixtures and Aggregate Names
# Group by all identifier columns and aggregate the unique opponent team names.
fixture_groups = df.groupby(['season_x', 'GW', 'fixture'])['opp_team_name'].unique()

# 3. Convert the aggregated series into a DataFrame, splitting the array of two names into two columns.
# The unique set of opponent names for a fixture are the two participating teams.
fixture_df = fixture_groups.apply(lambda x: pd.Series(x[:2]))

# Rename the new columns and reset the index to flatten the DataFrame.
fixture_df = fixture_df.rename(columns={0: 'team_1_name', 1: 'team_2_name'}).reset_index()

# # 4. Map IDs to Names
# # Use the dictionary created in step 1 to map the team names to their corresponding FPL IDs.
# fixture_df['team_1_id'] = fixture_df['team_1_name'].map(name_to_id_map)
# fixture_df['team_2_id'] = fixture_df['team_2_name'].map(name_to_id_map)

# 5. Select and reorder the final columns as requested
final_columns = [
    'season_x',
    'GW',
    'fixture',
    'team_1_name',
    'team_2_name',
    # 'team_1_id',
    # 'team_2_id'
]
fixture_df = fixture_df[final_columns]

In [22]:
fixture_df.head(20)

,season_x,GW,fixture,team_1_name,team_2_name
0,2016-17,1,1,Swansea,Burnley
1,2016-17,1,2,West Brom,Crystal Palace
2,2016-17,1,3,Spurs,Everton
3,2016-17,1,4,Leicester,Hull
4,2016-17,1,5,Sunderland,Man City
5,2016-17,1,6,Stoke,Middlesbrough
6,2016-17,1,7,Southampton,Watford
7,2016-17,1,8,Liverpool,Arsenal
8,2016-17,1,9,Man Utd,Bournemouth
9,2016-17,1,10,Chelsea,West Ham


In [28]:
df_merged = df.merge(
    fixture_df[['fixture', 'team_1_name', 'team_2_name', 'GW', 'season_x']],
    on=['fixture', 'GW', 'season_x'],
    how='left'
)
df_merged.head()

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,team_1_name,team_2_name
0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,...,0,0,0,0,55,False,0,1,Chelsea,West Ham
1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,...,1,0,0,0,60,True,0,1,Spurs,Everton
2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,...,2,0,0,0,80,True,0,1,Liverpool,Arsenal
3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,...,0,0,0,0,50,False,0,1,Southampton,Watford
4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,...,1,0,0,0,45,True,1,1,Stoke,Middlesbrough


In [30]:
# 2. Define the Conditional Fill Logic
def fill_missing_team_by_fixture(row):
    # If team_x is already present, return it.
    if pd.notna(row['team_x']):
        return row['team_x']

    # Identify the opponent name
    opponent = row['opp_team_name']

    # The missing team must be the non-opponent team among the two fixture teams (team_1_name and team_2_name)
    if row['team_1_name'] == opponent:
        # If team 1 is the opponent, the player's team is team 2
        return row['team_2_name']
    elif row['team_2_name'] == opponent:
        # If team 2 is the opponent, the player's team is team 1
        return row['team_1_name']
    else:
        # Fallback if names don't match
        return row['team_x']

In [33]:
# 3. Apply the fill logic
df_merged['team_x_fixed'] = df_merged.apply(fill_missing_team_by_fixture, axis=1)
# 4. Finalize the 'team_x' column by filling NaNs with the derived values
df_merged['team_x'] = df_merged['team_x'].fillna(df_merged['team_x_fixed'])

In [36]:
# 5. Clean up temporary columns and prepare final DataFrame
df_final = df_merged.drop(columns=['team_1_name', 'team_2_name', 'team_x_fixed'])

In [38]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96169 entries, 0 to 96168
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season_x           96169 non-null  object 
 1   name               96169 non-null  object 
 2   position           96169 non-null  object 
 3   team_x             96169 non-null  object 
 4   assists            96169 non-null  int64  
 5   bonus              96169 non-null  int64  
 6   bps                96169 non-null  int64  
 7   clean_sheets       96169 non-null  int64  
 8   creativity         96169 non-null  float64
 9   element            96169 non-null  int64  
 10  fixture            96169 non-null  int64  
 11  goals_conceded     96169 non-null  int64  
 12  goals_scored       96169 non-null  int64  
 13  ict_index          96169 non-null  float64
 14  influence          96169 non-null  float64
 15  kickoff_time       96169 non-null  object 
 16  minutes            961

In [41]:
cleaned_df = df_final.drop(['round', 'bps', 'creativity', 'influence' ,'threat', 'kickoff_time'], axis=1)
cleaned_df.head(5)

,season_x,name,position,team_x,assists,bonus,clean_sheets,element,fixture,goals_conceded,...,team_a_score,team_h_score,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,West Ham,0,0,0,454,10,0,...,1.0,2.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,MID,Everton,0,0,0,142,3,0,...,1.0,1.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,Arsenal,0,0,0,16,8,3,...,4.0,3.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,Watford,0,0,0,482,7,0,...,1.0,1.0,0,0,0,0,50,False,0,1
4,2016-17,Adam Forshaw,MID,Middlesbrough,0,0,0,286,6,1,...,1.0,1.0,1,0,0,0,45,True,1,1


In [42]:
# Shows the actual duplicated rows
cleaned_df[cleaned_df.duplicated()]

,season_x,name,position,team_x,assists,bonus,clean_sheets,element,fixture,goals_conceded,...,team_a_score,team_h_score,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW


In [43]:
cleaned_df.rename(columns={
    'total_points': 'GW_points',
    'element': 'player_id'
}, inplace=True)
cleaned_df.head()

,season_x,name,position,team_x,assists,bonus,clean_sheets,player_id,fixture,goals_conceded,...,team_a_score,team_h_score,GW_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,West Ham,0,0,0,454,10,0,...,1.0,2.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,MID,Everton,0,0,0,142,3,0,...,1.0,1.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,Arsenal,0,0,0,16,8,3,...,4.0,3.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,Watford,0,0,0,482,7,0,...,1.0,1.0,0,0,0,0,50,False,0,1
4,2016-17,Adam Forshaw,MID,Middlesbrough,0,0,0,286,6,1,...,1.0,1.0,1,0,0,0,45,True,1,1


In [63]:
cleaned_df['GW'] = cleaned_df['GW'].astype(int)
cleaned_df.sort_values(by=['season_x', 'player_id', 'GW'], inplace=True)
cleaned_df.head(10)

,season_x,name,position,team_x,assists,bonus,clean_sheets,player_id,fixture,goals_conceded,...,team_a_score,team_h_score,GW_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
75,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,8,4,...,4.0,3.0,0,0,0,0,65,True,0,1
2480,2016-17,Héctor Bellerín,DEF,Arsenal,0,3,1,6,13,0,...,0.0,0.0,9,-56304,4952,61256,65,False,0,2
4816,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,28,1,...,3.0,1.0,2,-14824,23529,38353,65,False,0,3
7366,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,31,1,...,1.0,2.0,2,-22447,29018,51465,65,True,0,4
7579,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,43,1,...,4.0,1.0,2,-34253,6258,40511,65,False,0,5
7792,2016-17,Héctor Bellerín,DEF,Arsenal,1,2,1,6,51,0,...,0.0,3.0,11,-64440,5049,69489,65,True,0,6
8005,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,1,6,61,0,...,1.0,0.0,6,50025,63674,13649,65,False,0,7
8219,2016-17,Héctor Bellerín,DEF,Arsenal,1,0,0,6,71,2,...,2.0,3.0,4,15926,36844,20918,65,True,0,8
8434,2016-17,Héctor Bellerín,DEF,Arsenal,0,1,1,6,81,0,...,0.0,0.0,7,39161,50283,11122,66,True,0,9
280,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,97,1,...,4.0,1.0,2,25971,31412,5441,66,False,0,10


In [65]:
cleaned_df['total_points'] = cleaned_df.groupby(['season_x', 'player_id'])['GW_points'].cumsum()

In [68]:
cleaned_df[cleaned_df['name'] == 'Héctor Bellerín']

,season_x,name,position,team_x,assists,bonus,clean_sheets,player_id,fixture,goals_conceded,...,team_h_score,GW_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,total_points
75,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,8,4,...,3.0,0,0,0,0,65,True,0,1,0
2480,2016-17,Héctor Bellerín,DEF,Arsenal,0,3,1,6,13,0,...,0.0,9,-56304,4952,61256,65,False,0,2,9
4816,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,28,1,...,1.0,2,-14824,23529,38353,65,False,0,3,11
7366,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,31,1,...,2.0,2,-22447,29018,51465,65,True,0,4,13
7579,2016-17,Héctor Bellerín,DEF,Arsenal,0,0,0,6,43,1,...,1.0,2,-34253,6258,40511,65,False,0,5,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66182,2021-22,Héctor Bellerín,DEF,Arsenal,0,0,0,9,349,0,...,1.0,0,-17,0,17,48,False,0,35,0
66990,2021-22,Héctor Bellerín,DEF,Arsenal,0,0,0,9,351,0,...,2.0,0,-22,0,22,48,True,0,36,0
66991,2021-22,Héctor Bellerín,DEF,Arsenal,0,0,0,9,218,0,...,3.0,0,-22,0,22,48,False,0,36,0
68914,2021-22,Héctor Bellerín,DEF,Arsenal,0,0,0,9,365,0,...,2.0,0,-28,0,28,48,False,0,37,0
